<a href="https://colab.research.google.com/github/hugosousa111/segmentacao_base_ph2/blob/main/pr%C3%A9_processamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configurações e Bibliotecas

In [ ]:
# Acessa o Drive
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
#importando bibliotecas
import matplotlib.pyplot as plt 
import numpy as np 
from scipy import signal as sg
import cv2 as cv
import warnings # só pra tirar os warnings
warnings.filterwarnings("ignore") 
import os
from IPython.display import clear_output
import pandas as pd
from collections import deque # filas ... 

# Funções Auxiliares

In [ ]:
def plota(image, text="Imagem", colorida=False):
    fig = plt.figure(figsize=(9,3), dpi=100)
    a = fig.add_subplot(1,3,1)
    a.axis('off')
    if not colorida:
        plt.imshow(image, cmap=plt.cm.gray)
    else:
        plt.imshow(image[:,:,::-1])  #img colorida
    a.set_title(text)
    plt.show()

def plota_save(image, caminho="", nome="name", text="Imagem",  colorida=False):
    dpi = 80

    height, width, nbands = image.shape

    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])

    # Hide spines, ticks, etc.
    ax.axis('off')

    # Display the image.
    if not colorida:
        ax.imshow(image, cmap=plt.cm.gray)
    else:
        ax.imshow(image[:,:,::-1])
    #ax.imshow(image, interpolation='nearest')

    # Ensure we're displaying with square pixels and the right extent.
    # This is optional if you haven't called `plot` or anything else that might
    # change the limits/aspect.  We don't need this step in this case.
    ax.set(xlim=[-0.5, width - 0.5], ylim=[height - 0.5, -0.5], aspect=1)

    fig.savefig('/content/drive/My Drive/PH2_TCC/'+caminho+'/'+nome, dpi=dpi, transparent=True)
    plt.show()

def just_save(image, caminho="", nome="name",  colorida=False):
    dpi = 80

    if not colorida:
        height, width = image.shape

        # What size does the figure need to be in inches to fit the image?
        figsize = width / float(dpi), height / float(dpi)

        # Create a figure of the right size with one axes that takes up the full figure
        fig = plt.figure(figsize=figsize)
        ax = fig.add_axes([0, 0, 1, 1])

        # Hide spines, ticks, etc.
        ax.axis('off')

        # Display the image.

        ax.imshow(image, cmap=plt.cm.gray)
    else:
        height, width, nbands = image.shape

        # What size does the figure need to be in inches to fit the image?
        figsize = width / float(dpi), height / float(dpi)

        # Create a figure of the right size with one axes that takes up the full figure
        fig = plt.figure(figsize=figsize)
        ax = fig.add_axes([0, 0, 1, 1])

        # Hide spines, ticks, etc.
        ax.axis('off')

        # Display the image.

        ax.imshow(image[:,:,::-1])
    #ax.imshow(image, interpolation='nearest')

    # Ensure we're displaying with square pixels and the right extent.
    # This is optional if you haven't called `plot` or anything else that might
    # change the limits/aspect.  We don't need this step in this case.
    ax.set(xlim=[-0.5, width - 0.5], ylim=[height - 0.5, -0.5], aspect=1)

    fig.savefig('/content/drive/My Drive/PH2_TCC/'+caminho+'/'+nome, dpi=dpi, transparent=True)
    #plt.show()
    plt.close(fig)
    
def salva_contexto(array, nome_array, j_d): 
    arq = open('/content/drive/My Drive/PH2_TCC/variaveis_'+j_d+'/'+nome_array+'.txt', 'w')
    np.savetxt(arq, array)
    arq.close()

In [ ]:
def get_names_images():
    filenames= os.listdir ("/content/drive/My Drive/PH2Dataset/PH2 Dataset images") # get all files' and folders' names in the current directory

    result = []
    for filename in filenames: # loop through all the files and folders
        if os.path.isdir(os.path.join(os.path.abspath("/content/drive/My Drive/PH2Dataset/PH2 Dataset images"), filename)): # check whether the current object is a folder or not
            result.append(filename)        
    result.sort()
    return result

imagens = get_names_images()
#imagens = imagens[0:30]
#imagens

In [ ]:
def plota_grafico(X, Y, segmentador, indice, tipo, Y2 = 0): 
    plt.figure(figsize=(10,5))
    if Y2 == 0: 
        plt.errorbar(X, Y, marker='s',  fmt='o',mfc='red', ecolor='gray', capsize=5);
        for x,y in zip(X,Y):
            label = "{:.4f}".format(y)
            plt.annotate(label, (x,y), textcoords="offset points", xytext=(20,3), ha='center')
    else: 
        plt.errorbar(X, Y,yerr=Y2, marker='s',  fmt='o',mfc='red', ecolor='gray', capsize=5);
        for x,y in zip(X,Y):
            label = "{:.4f}".format(y)
            plt.annotate(label, (x,y), textcoords="offset points", xytext=(20,3), ha='center')
        for x,y,z in zip(X,Y, Y2):
            label = "{:.2f}".format(y+z)
            plt.annotate(label, (x,y+z), textcoords="offset points", xytext=(15,3), ha='center') 
        for x,y,z in zip(X,Y, Y2):
            label = "{:.2f}".format(y-z)
            plt.annotate(label, (x,y-z), textcoords="offset points", xytext=(15,3), ha='center') 
    #plt.tick_params(axis='x', which='major', labelsize='10')
    
    plt.ylabel(indice)
    title = tipo+', Segmentador: '+segmentador+', Índice: '+indice
    plt.title(title)
    
    plt.margins(0.09,0)
    plt.xticks(rotation=0)
    plt.ylim(ymin=0, ymax=1.1)
    plt.grid(b='on')
    plt.show()

def gera_graficobar(X, melhores, inter, piores, segmentador, indice, melhor, pior):
    labels = X

    x = np.arange(len(labels))  # the label locations
    width = 0.27 # the width of the bars

    fig, ax = plt.subplots(figsize=(10, 5))
    rects1 = ax.bar(x - width, melhores, width, color='g', label='Melhores (>'+str(melhor)+')')
    rects2 = ax.bar(x, inter, width,label='Intermediários')
    rects3 = ax.bar(x + width, piores, width,color='r', label='Piores (<'+str(pior)+')')

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Amostras')
    title = 'Amostras por intervalo, segmentador: '+segmentador+', índice: '+indice
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    def autolabel(rects):
        """Attach a text label above each bar in *rects*, displaying its height."""
        for rect in rects:
            height = rect.get_height()
            ax.annotate('{}'.format(height),
                        xy=(rect.get_x() + rect.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom')
    autolabel(rects1)
    autolabel(rects2)
    autolabel(rects3)
    #fig.tight_layout()
    plt.margins(0.09,0)
    plt.xticks(rotation=0)
    plt.ylim(ymin=0, ymax=210)
    ax.set_axisbelow(True)
    plt.grid(b='on', color='gray', linestyle='--', linewidth=0.5)
    #plt.xlim(xmin=0, xmax=10)
    plt.show()


def melhor_pior(var, melhor, pior):
    melhores = len(var[var>melhor])
    piores = len(var[var<pior])
    intermediario = len(var) - melhores - piores

    return melhores,intermediario, piores

def maior_media_experimento(Y):
    if Y[0]>Y[1] and Y[0]>Y[2] and Y[0]>Y[3]:
        return 'P_P'
    elif Y[1]>Y[0] and Y[1]>Y[2] and Y[1]>Y[3]:
        return 'NP_P'
    elif Y[2]>Y[0] and Y[2]>Y[1] and Y[2]>Y[3]:
        return 'P_NP'
    else:
        return 'NP_NP'

def maior_melhor_experimento(Y):
    if Y[0]>Y[1] and Y[0]>Y[2] and Y[0]>Y[3]:
        return 'P_P'
    elif Y[1]>Y[0] and Y[1]>Y[2] and Y[1]>Y[3]:
        return 'NP_P'
    elif Y[2]>Y[0] and Y[2]>Y[1] and Y[2]>Y[3]:
        return 'P_NP'
    else:
        return 'NP_NP'

def plota_conjunto(var, min_max, z, segmentador, indice):
    if min_max == 'max':
        img_name = imagens[np.argmax(var)]
        print(indice+" -> ",np.max(var), "\n")
    else: 
        img_name = imagens[np.argmin(var)]
        print(indice+" -> ",np.min(var), "\n")
    print(img_name)

    imgOriginal = carrega_imagem_original(img_name, 1)
    plota(imgOriginal, text='Img Original', colorida=True)
    just_save(imgOriginal, caminho="outros", nome="imgOriginal",  colorida=True)

    if segmentador == 'Kmeans':
        if z == 'P_P':
            imgSegmentada = carrega_Resultados_Pre_kmeans_Pos(img_name)
        elif z == 'NP_P': 
            imgSegmentada = carrega_Resultados_NPre_kmeans_Pos(img_name)
        elif z == 'P_NP': 
            imgSegmentada = carrega_Resultados_Pre_kmeans_NPos(img_name)
        else: 
            imgSegmentada = carrega_Resultados_NPre_kmeans_NPos(img_name)
    elif segmentador == 'Otsu':
        if z == 'P_P':
            imgSegmentada = carrega_Resultados_Pre_Otsu_Pos(img_name)
        elif z == 'NP_P': 
            imgSegmentada = carrega_Resultados_NPre_Otsu_Pos(img_name)
        elif z == 'P_NP': 
            imgSegmentada = carrega_Resultados_Pre_Otsu_NPos(img_name)
        else: 
            imgSegmentada = carrega_Resultados_NPre_Otsu_NPos(img_name)
    elif segmentador == 'Binarização':
        if z == 'P_P':
            imgSegmentada = carrega_Resultados_Pre_binarizacao_Pos(img_name)
        elif z == 'NP_P': 
            imgSegmentada = carrega_Resultados_NPre_binarizacao_Pos(img_name)
        elif z == 'P_NP': 
            imgSegmentada = carrega_Resultados_Pre_binarizacao_NPos(img_name)
        else: 
            imgSegmentada = carrega_Resultados_NPre_binarizacao_NPos(img_name)
    elif segmentador == 'Binarização Gauss':
        if z == 'P_P':
            imgSegmentada = carrega_Resultados_Pre_binarizacao_adptativa_gauss_Pos(img_name)
        elif z == 'NP_P': 
            imgSegmentada = carrega_Resultados_NPre_binarizacao_adptativa_gauss_Pos(img_name)
        elif z == 'P_NP': 
            imgSegmentada = carrega_Resultados_Pre_binarizacao_adptativa_gauss_NPos(img_name)
        else: 
            imgSegmentada = carrega_Resultados_NPre_binarizacao_adptativa_gauss_NPos(img_name)
    elif segmentador == 'Binarização Mean':
        if z == 'P_P':
            imgSegmentada = carrega_Resultados_Pre_binarizacao_adptativa_mean_Pos(img_name)
        elif z == 'NP_P': 
            imgSegmentada = carrega_Resultados_NPre_binarizacao_adptativa_mean_Pos(img_name)
        elif z == 'P_NP': 
            imgSegmentada = carrega_Resultados_Pre_binarizacao_adptativa_mean_NPos(img_name)
        else: 
            imgSegmentada = carrega_Resultados_NPre_binarizacao_adptativa_mean_NPos(img_name)
    elif segmentador == 'Crescimento de Região':
        if z == 'P_P':
            imgSegmentada = carrega_Resultados_Pre_crescimento_de_regiao_Pos(img_name)
        elif z == 'NP_P': 
            imgSegmentada = carrega_Resultados_NPre_crescimento_de_regiao_Pos(img_name)
        elif z == 'P_NP': 
            imgSegmentada = carrega_Resultados_Pre_crescimento_de_regiao_NPos(img_name)
        else: 
            imgSegmentada = carrega_Resultados_NPre_crescimento_de_regiao_NPos(img_name)
    elif segmentador == '---':
        if z == 'Branco':
            imgSegmentada = carrega_Resultados_tudo_branco(img_name)
        elif z == 'Preta': 
            imgSegmentada = carrega_Resultados_tudo_preto(img_name)
        else: 
            imgSegmentada = carrega_Resultados_tudo_com_filtro(img_name)
    
    plota(imgSegmentada, text='Img Segmentada')
    just_save(imgSegmentada, caminho="outros", nome="imgSegmentada")

    imgLesion = carrega_imagem_lesion(img_name)
    plota(imgLesion, text='Img Especialista')
    just_save(imgLesion, caminho="outros", nome="imgLesion")


def gera_tudo(p_p,np_p,p_np,np_np, segmentador, indice, melhor=0.8, pior=0.35):
    X = ['P-P', 'NP-P', 'P-NP', 'NP-NP']
    Y = [np.mean(p_p),np.mean(np_p),np.mean(p_np),np.mean(np_np)]
    Y_std = [np.std(p_p),np.std(np_p),np.std(p_np),np.std(np_np)]
    Y_max = [np.max(p_p),np.max(np_p),np.max(p_np),np.max(np_np)]
    Y_min = [np.min(p_p),np.min(np_p),np.min(p_np),np.min(np_np)]

    #plota_grafico(X, Y, segmentador, indice, 'Média e Desvio Padrão', Y_std)
    
    #plota_grafico(X, Y_max, segmentador, indice, 'Máximo')

    #plota_grafico(X, Y_min, segmentador, indice, 'Mínimo')

    x_ = 25
    y_ = -3
    plt.figure(figsize=(10,5))
    plt.errorbar(X, Y_max, marker='^',  label='Melhor Caso', fmt='o',mfc='g', ecolor='gray', capsize=5);
    for x,y in zip(X,Y_max):
        label = "{:.4f}".format(y)
        plt.annotate(label, (x,y), textcoords="offset points", xytext=(x_,y_-1), ha='center')
    plt.errorbar(X, Y_min, marker='v',  label='Pior Caso', fmt='o',mfc='red', ecolor='gray', capsize=5);
    for x,y in zip(X,Y_min):
        label = "{:.4f}".format(y)
        plt.annotate(label, (x,y), textcoords="offset points", xytext=(x_,y_-1), ha='center')
    plt.errorbar(X, Y, marker='s',   label='Média', fmt='o', ecolor='gray', capsize=5);
    for x,y in zip(X,Y):
        label = "\u03BC = {:.4f}".format(y)
        plt.annotate(label, (x,y), textcoords="offset points", xytext=(x_+10,y_), ha='center')
    for x,y,z in zip(X,Y, Y_std):
        label = "(\u03C3 = {:.4f})".format(z)
        plt.annotate(label, (x,y), textcoords="offset points", xytext=(x_+11,y_-10), ha='center') 
    #for x,y,z in zip(X,Y, Y_std):
    #    label = "{:.4f}".format(y-z)
    #    plt.annotate(label, (x,y-z), textcoords="offset points", xytext=(x_,y_), ha='center') 
    #plt.tick_params(axis='x', which='major', labelsize='10')
    
    plt.ylabel(indice)
    title = "tipo"+', Segmentador: '+"segmentador"+', Índice: '+indice
    plt.title(title)
    
    plt.margins(0.4,0)
    plt.xticks(rotation=0)
    plt.ylim(ymin=-0.05, ymax=1.3)
    plt.xlim(xmin=-0.5, xmax=4)
    plt.grid(b='on', color='gray', linestyle='--', linewidth=0.5)
    plt.legend(loc='upper right')
    plt.title("")
    plt.savefig('/content/drive/My Drive/PH2_TCC/outros/'+segmentador+'_'+indice+'_T', dpi=80, transparent=True)
    plt.show()
    
    #fig,(ax1)=plt.subplots(1,1)
    #for i in np.arange(len(x)):
    #    ax1.errorbar(x[i],y[i],yerr=yerr[i],xerr=xerr[i],label=subs[i],ecolor='black',marker='o',ls='')
    #ax1.legend(loc='upper left', numpoints=1)


    m_p_p, i_p_p, p_p_p = melhor_pior(p_p, melhor, pior)
    m_np_p, i_np_p, p_np_p = melhor_pior(np_p, melhor, pior)
    m_p_np, i_p_np, p_p_np = melhor_pior(p_np, melhor, pior)
    m_np_np, i_np_np, p_np_np = melhor_pior(np_np, melhor, pior)

    melhores = np.concatenate((m_p_p, m_np_p, m_p_np, m_np_np), axis=None)
    inter = np.concatenate((i_p_p, i_np_p, i_p_np, i_np_np), axis=None)
    piores = np.concatenate((p_p_p, p_np_p, p_p_np, p_np_np), axis=None)

    gera_graficobar(X, melhores, inter, piores, segmentador, indice, melhor, pior)

    print('\n'+segmentador+' + ' +indice+': ')
    print("Maior Média: ")
    z = maior_media_experimento(Y)
    print(z)
    print("Maior número de melhores amostras: ")
    print(maior_melhor_experimento(melhores))

    if z == 'P_P':
        var = p_p
    elif z == 'NP_P': 
        var = np_p
    elif z == 'P_NP':
        var = p_np
    else:
        var = np_np

    print('\nPior caso: ')
    plota_conjunto(var, 'min', z, segmentador,  indice)
    
    print('\nMelhor caso: ')
    plota_conjunto(var, 'max', z, segmentador, indice)
    

In [ ]:
def carrega_imagem_original(image, color=0):
    img = cv.imread('/content/drive/My Drive/PH2Dataset/PH2 Dataset images/'+image+'/'+image+'_Dermoscopic_Image/'+image+'.bmp', color)
    return img

def carrega_imagem_lesion(image, color=0):
    img = cv.imread('/content/drive/My Drive/PH2Dataset/PH2 Dataset images/'+image+'/'+image+'_lesion/'+image+'_lesion.bmp', color)
    return img
    
def carrega_imagem_processada(image, color=0):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Pre_Processadas_Black_Hat/'+image+'.png', color)
    return img

def carrega_Resultados_NPre_kmeans_Pos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_NPre_kmeans_Pos/'+image+'.png', 0)
    return img

def carrega_Resultados_Pre_kmeans_Pos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_Pre_kmeans_Pos/'+image+'.png', 0)
    return img

def carrega_Resultados_Pre_kmeans_NPos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_Pre_kmeans_NPos/'+image+'.png', 0)
    return img

def carrega_Resultados_NPre_kmeans_NPos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_NPre_kmeans_NPos/'+image+'.png', 0)
    return img

def carrega_Resultados_NPre_Otsu_Pos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_NPre_Otsu_Pos/'+image+'.png', 0)
    return img

def carrega_Resultados_Pre_Otsu_Pos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_Pre_Otsu_Pos/'+image+'.png', 0)
    return img

def carrega_Resultados_Pre_Otsu_NPos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_Pre_Otsu_NPos/'+image+'.png', 0)
    return img

def carrega_Resultados_NPre_Otsu_NPos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_NPre_Otsu_NPos/'+image+'.png', 0)
    return img

def carrega_Resultados_NPre_binarizacao_Pos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_NPre_binarizacao_Pos/'+image+'.png', 0)
    return img

def carrega_Resultados_Pre_binarizacao_Pos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_Pre_binarizacao_Pos/'+image+'.png', 0)
    return img

def carrega_Resultados_Pre_binarizacao_NPos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_Pre_binarizacao_NPos/'+image+'.png', 0)
    return img

def carrega_Resultados_NPre_binarizacao_NPos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_NPre_binarizacao_NPos/'+image+'.png', 0)
    return img

def carrega_Resultados_tudo_branco(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_tudo_branco/'+image+'.png', 0)
    return img

def carrega_Resultados_tudo_preto(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_tudo_preto/'+image+'.png', 0)
    return img

def carrega_Resultados_tudo_com_filtro(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_tudo_com_filtro/'+image+'.png', 0)
    return img

def carrega_Resultados_NPre_binarizacao_adptativa_gauss_Pos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_NPre_binarizacao_adptativa_gauss_Pos/'+image+'.png', 0)
    return img

def carrega_Resultados_Pre_binarizacao_adptativa_gauss_Pos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_Pre_binarizacao_adptativa_gauss_Pos/'+image+'.png', 0)
    return img

def carrega_Resultados_Pre_binarizacao_adptativa_gauss_NPos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_Pre_binarizacao_adptativa_gauss_NPos/'+image+'.png', 0)
    return img

def carrega_Resultados_NPre_binarizacao_adptativa_gauss_NPos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_NPre_binarizacao_adptativa_gauss_NPos/'+image+'.png', 0)
    return img

def carrega_Resultados_NPre_binarizacao_adptativa_mean_Pos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_NPre_binarizacao_adptativa_mean_Pos/'+image+'.png', 0)
    return img

def carrega_Resultados_Pre_binarizacao_adptativa_mean_Pos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_Pre_binarizacao_adptativa_mean_Pos/'+image+'.png', 0)
    return img

def carrega_Resultados_Pre_binarizacao_adptativa_mean_NPos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_Pre_binarizacao_adptativa_mean_NPos/'+image+'.png', 0)
    return img

def carrega_Resultados_NPre_binarizacao_adptativa_mean_NPos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_NPre_binarizacao_adptativa_mean_NPos/'+image+'.png', 0)
    return img

def carrega_Resultados_NPre_crescimento_de_regiao_Pos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_NPre_crescimento_de_regiao_Pos/'+image+'.png', 0)
    return img

def carrega_Resultados_Pre_crescimento_de_regiao_Pos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_Pre_crescimento_de_regiao_Pos/'+image+'.png', 0)
    return img

def carrega_Resultados_Pre_crescimento_de_regiao_NPos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_Pre_crescimento_de_regiao_NPos/'+image+'.png', 0)
    return img

def carrega_Resultados_NPre_crescimento_de_regiao_NPos(image):
    img = cv.imread('/content/drive/My Drive/PH2_TCC/Resultados_NPre_crescimento_de_regiao_NPos/'+image+'.png', 0)
    return img


In [ ]:
df = pd.read_excel('/content/drive/My Drive/PH2Dataset/PH2_dataset.xlsx')
df.columns = ["Image Name","Histological Diagnosis","Common Nevus","Atypical Nevus", "Melanoma", "Asymmetry(0/1/2)", "Pigment Network(AT/T)", "Dots/Globules(A/AT/T)", "Streaks\n(A/P)","Regression Areas(A/P)", "Blue-Whitish Veil(A/P)", "White", "Red", "Light-Brown", "Dark-Brown", "Blue-Gray", "Black"]
df = df.iloc[12:,] #apagando legenda

def get_rotulo(img, df):
    if df["Common Nevus"][df["Image Name"] == img].values == 'X':
        return 0
    elif df["Atypical Nevus"][df["Image Name"] == img].values  == 'X':
        return 1
    else:
        return 2

def get_rotulo_name(img, df):
    if df["Common Nevus"][df["Image Name"] == img].values == 'X':
        return "Common Nevus"
    elif df["Atypical Nevus"][df["Image Name"] == img].values  == 'X':
        return "Atypical Nevus"
    else:
        return "Melanoma"

def get_histological(img, df):
    return df["Histological Diagnosis"][df["Image Name"] == img].values[0]

# Pré Processamento

In [ ]:
#src = carrega_imagem_original(imagens[0],1)
#plota(src, "Teste", True)
def pre_processamento_blackhat(src, plot=0):
    # Depilador
    # Convert the original image to grayscale
    grayScale = cv.cvtColor( src, cv.COLOR_RGB2GRAY )
    
    # Kernel for the morphological filtering
    kernel = cv.getStructuringElement(1,(15,15))

    # Perform the blackHat filtering on the grayscale image to find the hair countours
    blackhat = cv.morphologyEx(grayScale, cv.MORPH_BLACKHAT, kernel)
    
    # intensify the hair countours in preparation for the inpainting algorithm
    ret,thresh2 = cv.threshold(blackhat,10,255,cv.THRESH_BINARY)
    
    # inpaint the original image depending on the mask
    dst = cv.inpaint(src,thresh2,1,cv.INPAINT_TELEA)

    if plot==1: 
        plota(grayScale, "")
        just_save(grayScale, caminho="outros", nome="grayScale",  colorida=False)
    
        plota(blackhat, "")
        just_save(blackhat, caminho="outros", nome="blackhat",  colorida=False)

        plota(thresh2, "")
        just_save(thresh2, caminho="outros", nome="thresh2",  colorida=False)

    return dst

#dst = pre_processamento_blackhat(src)
#plota(dst, "Teste", True)

In [ ]:
def pre_processamento_array(imagens, tipo=0): 
    if tipo == 0:
        dir = "Pre_Processadas_Black_Hat"
        for img_nome in imagens:
            print(img_nome+' ...')
            img = carrega_imagem_original(img_nome, 1)
            #plota(img, "", True)
            img_pre = pre_processamento_blackhat(img)
            just_save(img_pre,dir,img_nome, True)
            #plota(img_pre, "", True)
    clear_output()

In [ ]:
# Pré-processamento
##imagens = get_names_images()
#pre_processamento_array(imagens) #############################